In [35]:
import os
import sys
from pyspark.sql import SparkSession, DataFrame # type: ignore
from pyspark.sql.functions import col, last, row_number, to_timestamp # type: ignore
from pyspark.sql.window import Window # type: ignore

In [36]:
spark = SparkSession.builder.appName("Testing_load_Silver").getOrCreate()
spark.sql("CREATE DATABASE IF NOT EXISTS datalake.silver")

DataFrame[]

In [39]:
bronze_table_gold = spark.table("datalake.bronze.gold")
bronze_table_gold.createOrReplaceTempView("bronze_table_gold")

print(bronze_table_gold.printSchema())
bronze_table_gold.show(5)

root
 |-- Date: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Vol: string (nullable = true)
 |-- Change: string (nullable = true)

None
+----------+-----+-----+-----+-----+------+------+
|      Date|Price| Open| High|  Low|   Vol|Change|
+----------+-----+-----+-----+-----+------+------+
|12/30/1993|391.9|393.3|393.9|391.0|29.64K| 0.03%|
|12/29/1993|391.8|386.6|392.5|384.8|18.71K| 0.85%|
|12/28/1993|388.5|389.7|390.0|387.9|39.20K|-0.15%|
|12/27/1993|389.1|388.1|389.4|387.7| 8.38K| 0.54%|
|12/23/1993|387.0|387.0|387.3|385.6| 6.91K|-0.05%|
+----------+-----+-----+-----+-----+------+------+
only showing top 5 rows



In [ ]:
df_temp = bronze_table_gold.select("Date", "Price", "Open", "High",  "Low")
df_temp = df_temp.withColumn("Date", to_timestamp(col("Date"), "MM/dd/yyyy").cast("date"))
df_temp = df_temp.withColumn("Price", col("Price").cast("float"))
df_temp = df_temp.withColumn("Open", col("Open").cast("float"))
df_temp = df_temp.withColumn("High", col("High").cast("float"))
df_temp = df_temp.withColumn("Low", col("Low").cast("float"))

print(df_temp.printSchema())
df_temp.show(5)

TypeError: cast() takes 2 positional arguments but 3 were given

In [43]:
df_temp = bronze_table_gold.select("Date", "Price", "Open", "High",  "Low")
df_temp.show(5)

+----------+-----+-----+-----+-----+
|      Date|Price| Open| High|  Low|
+----------+-----+-----+-----+-----+
|12/30/1993|391.9|393.3|393.9|391.0|
|12/29/1993|391.8|386.6|392.5|384.8|
|12/28/1993|388.5|389.7|390.0|387.9|
|12/27/1993|389.1|388.1|389.4|387.7|
|12/23/1993|387.0|387.0|387.3|385.6|
+----------+-----+-----+-----+-----+
only showing top 5 rows



In [47]:
df_temp = df_temp.withColumn("Date", to_timestamp("Date", "MM/dd/yyyy").cast("date"))

print(df_temp.printSchema())
df_temp.show(5)

root
 |-- Date: date (nullable = true)
 |-- Price: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)

None
+----------+-----+-----+-----+-----+
|      Date|Price| Open| High|  Low|
+----------+-----+-----+-----+-----+
|1993-12-30|391.9|393.3|393.9|391.0|
|1993-12-29|391.8|386.6|392.5|384.8|
|1993-12-28|388.5|389.7|390.0|387.9|
|1993-12-27|389.1|388.1|389.4|387.7|
|1993-12-23|387.0|387.0|387.3|385.6|
+----------+-----+-----+-----+-----+
only showing top 5 rows



In [18]:
silver_tbl_gold = spark.table("datalake.silver.gold")
silver_tbl_gold.createOrReplaceTempView("silver_tbl_gold")

silver_tbl_gold.show(5)

+---+----+-----+-----+-----+-----+
| ID|Date|Price| Open| High|  Low|
+---+----+-----+-----+-----+-----+
|  1|NULL|332.5|333.0|333.4|332.3|
|  2|NULL|329.6|329.1|329.7|328.3|
|  3|NULL|332.8|334.1|334.3|332.0|
|  4|NULL|445.3|446.8|449.0|443.6|
|  5|NULL|449.2|448.5|450.5|448.0|
+---+----+-----+-----+-----+-----+
only showing top 5 rows



In [19]:
silver_tbl_gold.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Price: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [1]:
spark.stop()